In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
import re
tqdm.pandas()

In [2]:
review = pd.read_csv('use_data/all_use.csv', encoding = 'utf-8-sig')

In [6]:
review.to_csv('use_data/all_use.csv', encoding = 'utf-8-sig')

In [ ]:
review_text = review[['review_text']]

def preprocessing_nl(review):
    review = review.apply(lambda x : x.lower())#모두 소문자로 변환
    review = review.apply(lambda row: nltk.word_tokenize(row))#단어 토큰화
    stop_words = stopwords.words('english')
    review = review.apply(lambda x: [word for word in x if word not in (stop_words)])#불용어 제거
    review = review.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x]) #표제어 추출
    review = review.apply(lambda x: [word for word in x if len(word) > 3]) #길이 3이하 제거
    
    return review

temp = preprocessing_nl(review_text['review_text'])

In [32]:
review['preprocessing_review'] = temp

In [33]:
review.to_csv('preprocessing_review.csv', encoding = 'utf-8-sig')

In [40]:
fake = pd.read_csv('preprocessing_fake_review.csv')

In [35]:
fake_text = fake[['fake_review_text']]

In [36]:
temp = preprocessing_nl(fake_text['fake_review_text'])

In [37]:
fake['preprocessing_review'] = temp

In [38]:
fake.to_csv('preprocessing_fake_review.csv', encoding = 'utf-8-sig')

In [43]:
fake['location_check'] = fake['fake_reviewer_loca'].apply(lambda x : loca_check(x))

In [48]:
fake['preprocessing_review'].iloc[0]

"['order', 'delivery', 'cancel', 'order', 'without', 'reason', 'half', 'hour', 'wait']"

In [26]:
fake_text = fake[['fake_review_text']]

In [27]:
fake_temp = preprocessing_nl(fake_text['fake_review_text'])

In [28]:
fake['preprocessing_text'] = fake_temp

In [29]:
fake.to_csv('preprocessing_fake.csv')

In [13]:
reviewer= pd.read_csv('reviewer_attribute_NY.csv', encoding = 'utf-8-sig')

In [15]:
reviewer['location']# = reviewer['location'].apply(lambda x : x.replace('From ', ''))

0                   New York, NY
1        Manhattan, New York, NY
2                     Ithaca, NY
3                   New York, NY
4                   New York, NY
                  ...           
80693             Alexandria, VA
80694               New York, NY
80695             Ottawa, Canada
80696            Stony Point, NY
80697                Hoboken, NJ
Name: location, Length: 80698, dtype: object

In [42]:
def loca_check(text):
    if type(text) != float and text.find('NY') != -1:
        return 0
    else:
        return 1

In [17]:
reviewer['location_check'] = reviewer['location'].apply(lambda x : loca_check(x))

In [22]:
reviewer = reviewer.drop(['Unnamed: 0', 'location'], axis = 1)

In [23]:
reviewer.to_csv('reviewer_use_attribute_NY.csv')

In [2]:
review = pd.read_csv('preprocessing_review.csv', encoding = 'utf-8-sig')

In [3]:
reviewer = pd.read_csv('reviewer_use_attribute_NY.csv', encoding = 'utf-8-sig')

In [4]:
reviewer = reviewer.rename(columns={'url': 'reviewer_url'})

In [6]:
NY_test = pd.merge(reviewer, review, on='reviewer_url')

In [10]:
NY_test = NY_test.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0.1', 'review_res', 'Unnamed: 0.1.1'], axis = 1)

In [12]:
NY_test['voting'] = NY_test['review_useful'] + NY_test['review_funny'] + NY_test['review_cool']

In [15]:
NY_test = NY_test.drop(['review_useful', 'review_funny', 'review_cool', 'review_date'], axis = 1)

In [17]:
NY_test.to_csv('NY_test.csv', encoding = 'utf-8-sig')